In [4]:
from datasets import load_dataset
import ollama
dataset = load_dataset("ag_news")
texts = dataset["train"]["text"]
labels = dataset["train"]["label"]
mapping = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
print("texts:", texts[:5])
print("labels:", labels[:5])

Using the latest cached version of the dataset since ag_news couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\solow\.cache\huggingface\datasets\ag_news\default\0.0.0\eb185aade064a813bc0b7f42de02595523103ca4 (last modified on Tue May 28 20:31:18 2024).


texts: ["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'Oil prices soar to all-time record, posing new menace to US economy

In [41]:
import ollama
from sklearn.metrics import accuracy_score
# Function to interact with the Ollama API
def chat_with_context(history):
    response = ollama.chat(model='llama3', messages=history)
    return response['message']['content']

# Initialize the chat history
chat_history = []
num_samples = 5

# Step 1: Define Task Description for AG News Dataset
task_description = "We need to predict the category of a given news article. The labels are 'World', 'Sports', 'Business', and 'Sci/Tech'."
instruction = "Classify each news article as 'World', 'Sports', 'Business', or 'Sci/Tech'."

generate_samples_request = (
    f"As an advanced language model you should create {num_samples} samples for the task outlined below.\n"
    "Generate samples that are likely to be correctly classified as 'World', 'Sports', 'Business', or 'Sci/Tech' and samples that might be misclassified according to the task instructions.\n\n"
    f"### Task Description:\n{task_description}\n\n"
    f"### Task Instructions:\n{instruction}\n\n"
    "### Requirements for Samples:\n"
    "1. Each sample must present a unique and intricate challenge.\n"
    "2. The complexity of the samples should be such that simply applying the given task instruction would likely lead to incorrect or incomplete results for some samples.\n"
    "3. The samples should cover a diverse range of scenarios within the scope of the task, avoiding repetition and predictability.\n"
    "4. Ensure that the samples, while challenging, remain realistic and pertinent to the task's context.\n"
    "Generate the samples keeping these requirements in mind.\n###"
)

chat_history.append({'role': 'user', 'content': task_description})
chat_history.append({'role': 'user', 'content': generate_samples_request})

samples_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': samples_response})
print("Generated Samples:")
print(samples_response)

# Step 2: Analyze Samples with Chain of Thought
analyze_samples_request = (
    f"Based on the following samples: {samples_response}\n"
    "Think step by step and derive general principles for classifying news articles into categories 'World', 'Sports', 'Business', or 'Sci/Tech'.\n"
    "Avoid focusing on specific details of the provided samples. Instead, develop broader, example-agnostic guidelines that can be applied universally to classify any news article. Conclude your analysis with clear, concise bullet points outlining:\n"
    "- The general characteristics that typically define each category.\n"
    "- Common mistakes that might lead to misclassifications and how to avoid them.\n"
    "- Guidelines under which circumstances each label should be predicted.\n"
    "These principles should help in accurately predicting the category of a news article based on its content without additional context."
)

chat_history.append({'role': 'user', 'content': analyze_samples_request})

analysis_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': analysis_response})
print("Analysis of Samples:")
print(analysis_response)

# Step 3: Generate Optimized Prompt
generate_prompt_request = (
    f"Based on the following analysis: {analysis_response}\nGenerate an optimized prompt for predicting "
    "whether a news article is 'World', 'Sports', 'Business', or 'Sci/Tech'. Ensure the model responds only with 'World', 'Sports', 'Business', or 'Sci/Tech'.\n\n"
    "### Requirements for Optimized Prompt:\n"
    "1. The prompt must include a clear description of the task and the labels.\n"
    "2. It should provide a comprehensive criteria for classifying news articles as 'World', 'Sports', 'Business', or 'Sci/Tech' based on the analysis.\n"
    "3. The prompt must ensure that the model responds strictly with 'World', 'Sports', 'Business', or 'Sci/Tech'.\n"
    "4. The prompt should help the model avoid common pitfalls and misclassifications identified during the analysis.\n"
    "5. Ensure the language is unambiguous and tailored to maximize the model's prediction accuracy."
    "6. Encourage the model to think step by step."
    "Respond with no other explanation but only the content of the prompt that is ready for the model to predict\n"
    "Prompt:"
)

chat_history.append({'role': 'user', 'content': generate_prompt_request})

optimized_prompt_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': optimized_prompt_response})
print("Optimized Prompt:")
print(optimized_prompt_response)

# Function to predict the label of a given news article
def get_prediction(text):
    prompt = (
        "Follow the guidelines of the prompt:\n"
        f"{optimized_prompt_response}\n\n"
        f"article:{text}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )
    response = ollama.generate(model='llama3', prompt=prompt)

    # Extract the predicted label from the response
    prediction = response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    # if prediction is not valid, perform the prediction again
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = get_prediction(text)
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i+1} samples out of {len(texts)}")
    if i == 10000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Generated Samples:
Here are five sample news articles for the task:

**Sample 1:**
Title: "New Study Reveals Surprising Impact of Climate Change on Global Food Production"
Content: A recent study published in a leading scientific journal has found that climate change is affecting global food production in unexpected ways. The researchers analyzed data from over 100 countries and discovered that certain regions are experiencing increased crop yields due to warmer temperatures, while others are seeing significant declines. Experts warn that these findings highlight the need for more precise climate modeling and adaptation strategies.

**Classification Challenge:** This sample might be misclassified as 'Sci/Tech' because of the scientific study and terminology used, but it could also be correctly classified as 'World' since the topic is global food production and climate change has international implications.

**Sample 2:**
Title: "Liverpool FC Signs Record-Breaking Transfer Deal for Star

# zero shot COT Prompt

In [35]:
# Function to predict the label of a given news article
def get_prediction(text):
    prompt = (
        f"Predict the news article into one of following categories: ('World', 'Sports', 'Business', or 'Sci/Tech')\n\n"
        "Let's think step by step\n"
        f"article:{text}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )
    response = ollama.generate(model='llama3', prompt=prompt)

    # Extract the predicted label from the response
    prediction = response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = "Invalid Prediction"
    
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i+1} samples out of {len(texts)}")
    if i == 10000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Predicted Label: Business

Predicted 1 samples out of 120000
Predicted Label: Business

Predicted Label: Business

Predicted 1001 samples out of 120000
Predicted 2001 samples out of 120000
Predicted 3001 samples out of 120000
Predicted 4001 samples out of 120000
Predicted 5001 samples out of 120000
Predicted 6001 samples out of 120000
Predicted 7001 samples out of 120000
Predicted 8001 samples out of 120000
Predicted 9001 samples out of 120000
Predicted 10001 samples out of 120000
Model Accuracy: 0.74


# Few show Prompt

In [36]:
few_shot_texts = texts[:3]
few_shot_labels = labels[:3]

# Function to predict the label of a given news article
def get_prediction(text):
    prompt = (
        f"Here are few news articles and their categories:\n\n"
        f"1. {few_shot_texts[0]} - {mapping[few_shot_labels[0]]}\n"
        f"2. {few_shot_texts[1]} - {mapping[few_shot_labels[1]]}\n"
        f"3. {few_shot_texts[2]} - {mapping[few_shot_labels[2]]}\n\n"
        "Based on these examples, predict the category of the following news article into one of the following categories: ('World', 'Sports', 'Business', or 'Sci/Tech')\n\n"
        f"article:{text}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )
    response = ollama.generate(model='llama3', prompt=prompt)

    # Extract the predicted label from the response
    prediction = response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = "Invalid Prediction"
    
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i+1} samples out of {len(texts)}")
    if i == 10000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Predicted Label: Business

Predicted 1 samples out of 120000
Predicted Label: Business

Predicted Label: Business

Predicted 1001 samples out of 120000
Predicted 2001 samples out of 120000
Predicted 3001 samples out of 120000
Predicted 4001 samples out of 120000
Predicted 5001 samples out of 120000
Predicted 6001 samples out of 120000
Predicted 7001 samples out of 120000
Predicted 8001 samples out of 120000
Predicted 9001 samples out of 120000
Predicted 10001 samples out of 120000
Model Accuracy: 0.74


# Basic Prompt

In [5]:
# Function to predict the label of a given news article
def get_prediction(text):
    prompt = (
        f"Predict the news article into one of following categories: ('World', 'Sports', 'Business', or 'Sci/Tech')\n\n"
        f"article:{text}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )
    response = ollama.generate(model='llama3', prompt=prompt)

    # Extract the predicted label from the response
    prediction = response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = "Invalid Prediction"
    
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i+1} samples out of {len(texts)}")
    if i == 1000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Predicted Label: Business

Predicted 1 samples out of 120000
Predicted Label: Business

Predicted Label: Business

Predicted 1001 samples out of 120000


NameError: name 'accuracy_score' is not defined

# Two step answer retrival

In [45]:
import ollama
from sklearn.metrics import accuracy_score
# Function to interact with the Ollama API
def chat_with_context(history):
    response = ollama.chat(model='llama3', messages=history)
    return response['message']['content']

# Initialize the chat history
chat_history = []
num_samples = 5

# Step 1: Define Task Description for AG News Dataset
task_description = "We need to predict the category of a given news article. The labels are 'World', 'Sports', 'Business', and 'Sci/Tech'."
instruction = "Classify each news article as 'World', 'Sports', 'Business', or 'Sci/Tech'."

generate_samples_request = (
    f"As an advanced language model you should create {num_samples} samples for the task outlined below.\n"
    "Generate samples that are likely to be correctly classified as 'World', 'Sports', 'Business', or 'Sci/Tech' and samples that might be misclassified according to the task instructions.\n\n"
    f"### Task Description:\n{task_description}\n\n"
    f"### Task Instructions:\n{instruction}\n\n"
    "### Requirements for Samples:\n"
    "1. Each sample must present a unique and intricate challenge.\n"
    "2. The complexity of the samples should be such that simply applying the given task instruction would likely lead to incorrect or incomplete results for some samples.\n"
    "3. The samples should cover a diverse range of scenarios within the scope of the task, avoiding repetition and predictability.\n"
    "4. Ensure that the samples, while challenging, remain realistic and pertinent to the task's context.\n"
    "Generate the samples keeping these requirements in mind.\n###"
)

chat_history.append({'role': 'user', 'content': task_description})
chat_history.append({'role': 'user', 'content': generate_samples_request})

samples_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': samples_response})
print("Generated Samples:")
print(samples_response)

# Step 2: Analyze Samples with Chain of Thought
analyze_samples_request = (
    f"Based on the following samples: {samples_response}\n"
    "Think step by step and derive general principles for classifying news articles into categories 'World', 'Sports', 'Business', or 'Sci/Tech'.\n"
    "Avoid focusing on specific details of the provided samples. Instead, develop broader, example-agnostic guidelines that can be applied universally to classify any news article. Conclude your analysis with clear, concise bullet points outlining:\n"
    "- The general characteristics that typically define each category.\n"
    "- Common mistakes that might lead to misclassifications and how to avoid them.\n"
    "- Guidelines under which circumstances each label should be predicted.\n"
    "These principles should help in accurately predicting the category of a news article based on its content without additional context."
)

chat_history.append({'role': 'user', 'content': analyze_samples_request})

analysis_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': analysis_response})
print("Analysis of Samples:")
print(analysis_response)

# Step 3: Generate Optimized Prompt
generate_prompt_request = (
    f"Based on the following analysis: {analysis_response}\nGenerate an optimized prompt for predicting "
    "whether a news article is 'World', 'Sports', 'Business', or 'Sci/Tech'. Ensure the model responds only with 'World', 'Sports', 'Business', or 'Sci/Tech'.\n\n"
    "### Requirements for Optimized Prompt:\n"
    "1. The prompt must include a clear description of the task and the labels.\n"
    "2. It should provide a comprehensive criteria for classifying news articles as 'World', 'Sports', 'Business', or 'Sci/Tech' based on the analysis.\n"
    "3. The prompt should help the model avoid common pitfalls and misclassifications identified during the analysis.\n"
    "4. Ensure the language is unambiguous and tailored to maximize the model's prediction accuracy."
    "5. Encourage the model to think step by step."
    "Respond with no other explanation but only the content of the prompt that is ready for the model to predict\n"
    "Prompt:"
)

chat_history.append({'role': 'user', 'content': generate_prompt_request})

optimized_prompt_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': optimized_prompt_response})
print("Optimized Prompt:")
print(optimized_prompt_response)

# Function to predict the label of a given news article
def get_prediction(text):
    # First part: Get a detailed response following the general guidelines of the prompt
    initial_prompt = (
        "Follow the guidelines of the prompt:\n"
        f"{optimized_prompt_response}\n\n"
        f"Article: {text}\n"
        "### Initial Analysis:\n"
        "Provide your detailed analysis and suggest a category based on the content of the article."
    )
    
    initial_response = ollama.generate(model='llama3', prompt=initial_prompt)
    detailed_analysis = initial_response['response'].strip()

    # Second part: Narrow down to just the predicted label
    final_prompt = (
        "Based on the detailed analysis, respond with only the category name:\n"
        f"{detailed_analysis}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )

    final_response = ollama.generate(model='llama3', prompt=final_prompt)
    prediction = final_response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    # if prediction is not valid, perform the prediction again
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = get_prediction(text)
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i+1} samples out of {len(texts)}")
    if i == 1000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Generated Samples:
Here are 5 samples for the task:

**Sample 1:**
Title: "New Study Reveals Surprising Link Between Climate Change and Stock Market Performance"
Text: A recent study published in a reputable scientific journal found that climate-related events have a significant impact on stock market fluctuations. The research suggests that investors should consider incorporating environmental, social, and governance (ESG) factors into their portfolio to mitigate risks.
**Category:** This sample is likely to be correctly classified as 'Sci/Tech', but might be misclassified as 'Business' or 'World'.

**Sample 2:**
Title: "Real Madrid Wins Champions League Title in Thrilling Finale"
Text: Real Madrid defeated Liverpool 3-1 in the UEFA Champions League final, with a late goal securing their record-breaking 13th title. The victory cements their status as one of the most successful clubs in the world.
**Category:** This sample is likely to be correctly classified as 'Sports'.

**Sample 3:*

In [7]:
print(optimized_prompt_response)

NameError: name 'optimized_prompt_response' is not defined